## Initialization

In [36]:
!pip install keras-tuner --upgrade
# !pip uninstall requests

In [1]:
# data processing
import numpy as np
import pandas as pd 
from collections import defaultdict

# data visualization
import seaborn as sns
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style
sns.set()
import urllib.request


import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
import keras_tuner as kt
from tensorflow import keras

# Notebook auto reloads code. (Ref: http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython)
%load_ext autoreload
%autoreload 2

C:\Users\free_\anaconda3\envs\napari-env\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\free_\anaconda3\envs\napari-env\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\free_\anaconda3\envs\napari-env\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
%pwd

'C:\\Users\\free_\\OneDrive\\Documents\\GitHub\\recreating-residual-attention-network\\nbks'

In [2]:
# If using Google Colab
from google.colab import drive
BASE_PATH = '/content/drive'
drive.mount(BASE_PATH)

# change directory
import os
PROJECT_PATH = os.path.join(BASE_PATH, "MyDrive", "2021-09 Fall Semester", "ECBM 4040 Neural Network Deep Learning", "Project", "recreating-residual-attention-network")
os.chdir(PROJECT_PATH)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#Jacob's path
%cd .. 

C:\Users\free_\OneDrive\Documents\GitHub\recreating-residual-attention-network


In [4]:
# Import created modules
from src.models.ResidualAttentionNetworkAugment import ResidualAttentionNetwork
from src.utils import generate_data

# Modelling

Code for fixing an error with downloading the dataset (SSL Error) on Jupyter notebook when running locally.

In [13]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context

In [5]:
train_dataset, validation_dataset, test_dataset = generate_data.get_cifar10_tensorslices()

In [12]:
# Import created modules
from src.models.ResidualAttentionNetwork import ResidualAttentionNetwork
from src.utils import generate_data

In [13]:
x_train, y_train, x_test, y_test, datagen = generate_data.get_cifar10(rotation_range=20,
                                                                      width_shift_range=0.2,
                                                                      height_shift_range=0.2,
                                                                      horizontal_flip=True,
                                                                      validation_split=0.2)

In [6]:
train_iter_im, train_iter_label = next(iter(train_dataset))


In [10]:
INPUT_SHAPE = train_iter_im.shape[1:]
NUM_CLASS = train_iter_label.shape[1]
BATCH_SIZE = 128
N_EPOCH = 3
print(f'Input Shape: {INPUT_SHAPE}')
print(f'NUM_CLASS: {NUM_CLASS}')

Input Shape: (32, 32, 3)
NUM_CLASS: 10


In [14]:
INPUT_SHAPE = x_train.shape[1:]
NUM_CLASS = y_train.shape[1]
BATCH_SIZE = 64
N_EPOCH = 3

In [15]:
ran_model = ResidualAttentionNetwork()
inputs = tf.keras.Input((32, 32, 3))
ran_model(inputs)
ran_model.summary()

Model: "residual_attention_network_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_85 (Conv2D)           multiple                  864       
_________________________________________________________________
batch_normalization_66 (Batc multiple                  128       
_________________________________________________________________
re_lu_66 (ReLU)              multiple                  0         
_________________________________________________________________
residual_unit_20 (ResidualUn multiple                  19520     
_________________________________________________________________
attention_module_2 (Attentio multiple                  347712    
_________________________________________________________________
residual_unit_30 (ResidualUn multiple                  96384     
_________________________________________________________________
attention_module_3 (Attentio multiple 

In [11]:
model = ran_model

model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss=tf.keras.losses.CategoricalCrossentropy(), 
              metrics=['accuracy'])


history = model.fit(train_dataset, 
                    validation_data=validation_dataset,
                    epochs=N_EPOCH, verbose=1)

Epoch 1/3
625/625 [==============================] - 495s 781ms/step - loss: 1.8644 - accuracy: 0.3039 - val_loss: 1.8104 - val_accuracy: 0.3475
Epoch 2/3
625/625 [==============================] - 578s 925ms/step - loss: 1.7065 - accuracy: 0.3717 - val_loss: 2.0704 - val_accuracy: 0.2903
Epoch 3/3
625/625 [==============================] - 677s 1s/step - loss: 1.6014 - accuracy: 0.4152 - val_loss: 2.0208 - val_accuracy: 0.3537


In [ ]:
model = ran_model

model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss=tf.keras.losses.CategoricalCrossentropy(), 
              metrics=['accuracy'])


history = model.fit(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE, subset='training'), 
                    validation_data=datagen.flow(x_train, y_train, batch_size=BATCH_SIZE, subset='validation'),
                    epochs=N_EPOCH, verbose=1)

Epoch 1/3
500/500 [==============================] - 599s 1s/step - loss: 1.8632 - accuracy: 0.2951 - val_loss: 2.3271 - val_accuracy: 0.2371
Epoch 2/3
270/500 [===============>..............] - ETA: 5:56 - loss: 1.6187 - accuracy: 0.3945

# Hyper Parameter Tuning

In [18]:
n_epochs = 20
model = ResidualAttentionNetwork()

model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss=tf.keras.losses.CategoricalCrossentropy(), 
              metrics=['accuracy'])


history = model.fit(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE, subset='training'), 
                    validation_data=datagen.flow(x_train, y_train, batch_size=BATCH_SIZE, subset='validation'),
                    epochs=n_epochs, verbose=1)

Epoch 1/20
313/313 [==============================] - 42s 115ms/step - loss: 1.9136 - accuracy: 0.2836 - val_loss: 3.5685 - val_accuracy: 0.1610
Epoch 2/20
313/313 [==============================] - 36s 115ms/step - loss: 1.5600 - accuracy: 0.4280 - val_loss: 1.6824 - val_accuracy: 0.3948
Epoch 3/20
313/313 [==============================] - 35s 113ms/step - loss: 1.4135 - accuracy: 0.4818 - val_loss: 1.8058 - val_accuracy: 0.3792
Epoch 4/20
313/313 [==============================] - 35s 112ms/step - loss: 1.3094 - accuracy: 0.5275 - val_loss: 1.7367 - val_accuracy: 0.4007
Epoch 5/20
313/313 [==============================] - 35s 113ms/step - loss: 1.2209 - accuracy: 0.5610 - val_loss: 1.3163 - val_accuracy: 0.5300
Epoch 6/20
313/313 [==============================] - 35s 112ms/step - loss: 1.1521 - accuracy: 0.5885 - val_loss: 1.3805 - val_accuracy: 0.5197
Epoch 7/20
313/313 [==============================] - 35s 113ms/step - loss: 1.0775 - accuracy: 0.6175 - val_loss: 1.6141 - val_ac

In [19]:
def model_builder(hp):
    ran_model = ResidualAttentionNetwork()
    inputs = tf.keras.Input((32, 32, 3))
    ran_model(inputs)
    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4,1e-5])
    ran_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate), 
              loss=tf.keras.losses.CategoricalCrossentropy(), 
              metrics=['accuracy'])
    return ran_model

In [47]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=25,
                     factor=3,
                     directory='NERD_dir',
                     project_name='NERD_Final_Project')

INFO:tensorflow:Reloading Oracle from existing project NERD_dir\NERD_Final_Project\oracle.json
INFO:tensorflow:Reloading Tuner from NERD_dir\NERD_Final_Project\tuner0.json


In [48]:
#create early stopping callback
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)


In [49]:

# history = model.fit(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE, subset='training'), 
#                     validation_data=datagen.flow(x_train, y_train, batch_size=BATCH_SIZE, subset='validation'),
#                     epochs=n_epochs, verbose=1)

In [50]:
# tuner.search(x_train, y_train, epochs=50, validation_data=(x_test, y_test), callbacks=[stop_early])
tuner.search(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE, subset='training'), epochs=50, validation_data=datagen.flow(x_train, y_train, batch_size=BATCH_SIZE, subset='validation'), callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 5 Complete [00h 01m 56s]
val_accuracy: 0.4643999934196472

Best val_accuracy So Far: 0.4643999934196472
Total elapsed time: 00h 05m 47s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal learning rate for the optimizer
is 0.001.



In [51]:
best_model = tuner.get_best_models()[0]


In [53]:
tuner.results_summary(num_trials=10)


Results summary
Results in NERD_dir\NERD_Final_Project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
learning_rate: 0.001
tuner/epochs: 3
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.4643999934196472
Trial summary
Hyperparameters:
learning_rate: 0.0001
tuner/epochs: 3
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.30809998512268066
Trial summary
Hyperparameters:
learning_rate: 0.01
tuner/epochs: 3
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.2904999852180481
Trial summary
Hyperparameters:
learning_rate: 1e-05
tuner/epochs: 3
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.15139999985694885


In [54]:
n_epochs = 75

model = ran_model

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), 
              loss=tf.keras.losses.CategoricalCrossentropy(), 
              metrics=['accuracy'])

history = model.fit(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE, subset='training'), 
                    validation_data=datagen.flow(x_train, y_train, batch_size=BATCH_SIZE, subset='validation'),
                    epochs=n_epochs, verbose=1)
best_model = model

Epoch 1/75
313/313 [==============================] - 42s 116ms/step - loss: 0.9126 - accuracy: 0.6770 - val_loss: 1.2114 - val_accuracy: 0.5960
Epoch 2/75
313/313 [==============================] - 35s 112ms/step - loss: 0.8571 - accuracy: 0.6993 - val_loss: 1.1214 - val_accuracy: 0.6190
Epoch 3/75
313/313 [==============================] - 35s 112ms/step - loss: 0.8262 - accuracy: 0.7107 - val_loss: 1.2021 - val_accuracy: 0.6079
Epoch 4/75
313/313 [==============================] - 35s 111ms/step - loss: 0.7845 - accuracy: 0.7255 - val_loss: 1.1659 - val_accuracy: 0.6148
Epoch 5/75
313/313 [==============================] - 35s 112ms/step - loss: 0.7568 - accuracy: 0.7365 - val_loss: 0.8808 - val_accuracy: 0.7013
Epoch 6/75
313/313 [==============================] - 35s 113ms/step - loss: 0.7281 - accuracy: 0.7452 - val_loss: 0.8760 - val_accuracy: 0.7029
Epoch 7/75
313/313 [==============================] - 35s 112ms/step - loss: 0.7098 - accuracy: 0.7527 - val_loss: 0.9100 - val_ac

# Save model

In [64]:
#model filename to save
from datetime import datetime
#include training params in model name
 
model_filename = str(datetime.now) + '_model_' + 'EPOCHS_'+ str(n_epochs) + str(BATCH_SIZE)
model_filepath = 'C:\\Users\\JacobNye\\Documents\\GitHub\\recreating-residual-attention-network\\data\\models\\'
#model file path default is data

In [65]:
#Save Model 
best_model.save(model_filepath)

C:\intelpython3\envs\napari-env\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: C:\Users\JacobNye\Documents\GitHub\recreating-residual-attention-network\data\models\assets


INFO:tensorflow:Assets written to: C:\Users\JacobNye\Documents\GitHub\recreating-residual-attention-network\data\models\assets


In [66]:
#test loading model
from tensorflow import keras
model = keras.models.load_model(model_filepath)
